## Configurações Iniciais

Primeiramente, fazer o ajuste no path do Python para a raíz do projeto:

In [1]:
import os
os.chdir('..')

Importar a biblioteca ```Morphosyntax e Syntax``` para fazer o processamento no corpora:

In [2]:
from nlputils.morphosyntax.tagger import Tagger
from nlputils.syntax.parser import Parser

### Agora vamos pegar as informações que estão no corpara e criar uma lista de senteças

Vamos transformar as senteças de cada corpus na seguinte estrutura de dados:
* {“verbo lematizado1”: [(Sujeito1, Objeto1), (Sujeito2, None), ..., (Sujeiton, Objeton)], ..., }

In [3]:
import os
from nlputils.lexical.normalizer import Normalizer

normalizer = Normalizer()

corpora_path = 'data/corpora'
files_telefonia = os.listdir('{}/telefonia/'.format(corpora_path))
files_telefonia = ['{}/telefonia/{}'.format(corpora_path,f) for f in files_telefonia if f != '.DS_Store']
files_saude = os.listdir('{}/saude/'.format(corpora_path))
files_saude = ['{}/saude/{}'.format(corpora_path,f) for f in files_saude if f != '.DS_Store']

corpus_lines_telefonia = []
for file in files_telefonia:
    with open(file, "r") as text_file:
        lines = text_file.readlines()
        corpus_lines_telefonia.extend(lines)
        
corpus_lines_saude = []
for file in files_saude:
    with open(file, "r") as text_file:
        lines = text_file.readlines()
        corpus_lines_saude.extend(lines)

sentences_telefonia = [normalizer.tokenize_sentences(corpus) for corpus in corpus_lines_telefonia]
sentences_saude = [normalizer.tokenize_sentences(corpus) for corpus in corpus_lines_saude]

Vamos utilizar a biblioteca spacy

In [5]:
import spacy

nlp = spacy.load('pt_core_news_sm')
p = Parser()
data = {}
SO = []
for corpus_sentence_telefonia in sentences_telefonia:
    for sentence in corpus_sentence_telefonia:
        svo_s = p.get_SVO(sentence)
        for svo in svo_s:
            SO.append((svo[0].text, svo[2].text))
            data[svo[1].lemma_] = SO
        SO = []

Logo abaixo está o resultado do dicionário:

In [6]:
import pprint

pp = pprint.PrettyPrinter(indent=2)

pp.pprint(data)

{ 'Vai tomar': [('°', '#')],
  'abandonar': [ ('Na', 'perceptível'),
                 ('imagem', 'perceptível'),
                 ('parte', 'manter'),
                 ('empresas', 'd')],
  'acabar': [('linha', 'mudar'), ('Samsung', 'mudar')],
  'acabar posicionar': [('se', 'difícil'), ('Apple', 'ultra')],
  'aceito': [ ('consumidor', 'pagar'),
              ('situação', 'dramática'),
              ('se', 'dramática')],
  'achar': [('vc', 'caro')],
  'acontecer': [('que', 'consigo'), ('que', 'câmera')],
  'acordar': [('mercado', 'abalado')],
  'acreditar': [('problema', 'iOS'), ('que', 'antigos')],
  'acumular': [('no', 'dos')],
  'aderir': [('Samsung', 'tambem')],
  'adiantar': [('tiro', 'atualizar'), ('tiro', 'adquiri')],
  'adotam': [('Huawei', 'problemasApós'), ('ZTE', 'problemasApós')],
  'adquirir': [('quem', 'importado')],
  'afirmar': [('Tim', 'Apple"Após atingir')],
  'ajudar': [('enriquecimento', 'enriquecer')],
  'alar': [('que', 'proibir')],
  'alertar': [('Especialistas', 

### Agora vamos fazer as seguintes estatísticas em cima do dicionário ```data```:
* Qual verbo tem a maior lista de sujeitos e objetos

In [7]:
greatest = [[len(data[d]), d] for d in data]
greatest = max(greatest)
greatest

[7, 'usar']

* Há algum verbo sem objetos? Mostre alguns.

In [8]:
smaller = [[len(data[d]), d] for d in data]
smaller = min(smaller)
smaller

[1, 'Vai tomar']

Não existe verbo sem objetos

### Sistema básico de perguntas e respostas utilizando o dicionário gerado acima

In [ ]:
question = input('Digite uma pergunta: ')

tagged_text = nlp(question)
verb = ''
for token in tagged_text:
    if token.pos_ == 'VERB':
        if token.lemma_ in data.keys():
            verb = token.lemma_
try:
    for text in data[verb]:
        print('Resposta: {} {} {}'.format(text[0], verb, text[1]))
except KeyError:
    print('A base de dados do dicionário não consegue responder está pergunta')
    